In [ ]:
!pip install transformers datasets torch torchmetrics scikit-learn pandas numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 46.4 MB/s eta 0:00:00


In [ ]:
import zipfile
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

# Extracting the dataset
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('goemotions')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [ ]:
# Load the dataset with correct column structure
train_df = pd.read_csv('goemotions/data/train.tsv', sep='\t', header=None)
dev_df = pd.read_csv('goemotions/data/dev.tsv', sep='\t', header=None)
test_df = pd.read_csv('goemotions/data/test.tsv', sep='\t', header=None)

print(f"Train shape: {train_df.shape}")
print(f"Dev shape: {dev_df.shape}")
print(f"Test shape: {test_df.shape}")

# Set correct column names
columns = ['text', 'emotion_ids', 'comment_id']
train_df.columns = columns
dev_df.columns = columns
test_df.columns = columns

print(f"\nColumns set to: {columns}")
print("\nFirst few rows of train:")
print(train_df.head())

Train shape: (43410, 3)
Dev shape: (5426, 3)
Test shape: (5427, 3)

Columns set to: ['text', 'emotion_ids', 'comment_id']

First few rows of train:
                                                text emotion_ids comment_id
0  My favourite food is anything I didn't have to...          27    eebbqej
1  Now if he does off himself, everyone will thin...          27    ed00q6i
2                     WHY THE FUCK IS BAYLESS ISOING           2    eezlygj
3                        To make her feel threatened          14    ed7ypvh
4                             Dirty Southern Wankers           3    ed0bdzj


In [ ]:
# Define emotion labels (from GoEmotions paper)
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
import torch
import numpy as np

# Convert labels to float32 instead of long
def preprocess_emotions(emotion_ids_str):
    try:
        if pd.isna(emotion_ids_str) or emotion_ids_str == '':
            return [0.0] * len(emotion_labels)  # Use float zeros

        emotion_ids = [int(id) for id in str(emotion_ids_str).split(',')]
        multi_hot = [1.0 if i in emotion_ids else 0.0 for i in range(len(emotion_labels))]  # Use float values
        return multi_hot
    except Exception as e:
        print(f"Error processing emotion_ids: {emotion_ids_str}, error: {e}")
        return [0.0] * len(emotion_labels)  # Return float zeros

# Apply preprocessing with float labels
train_df['labels'] = train_df['emotion_ids'].apply(preprocess_emotions)
dev_df['labels'] = dev_df['emotion_ids'].apply(preprocess_emotions)
test_df['labels'] = test_df['emotion_ids'].apply(preprocess_emotions)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
dev_dataset = Dataset.from_pandas(dev_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

# Tokenization function
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

    # Convert labels to float32
    labels = [torch.tensor(label, dtype=torch.float32) for label in examples['labels']]
    tokenized['labels'] = labels

    return tokenized

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_dev = dev_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_dev.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print("Datasets tokenized successfully!")
print(f"Sample labels type: {type(tokenized_train[0]['labels'])}")
print(f"Sample labels dtype: {tokenized_train[0]['labels'].dtype}")

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Datasets tokenized successfully!
Sample labels type: <class 'torch.Tensor'>
Sample labels dtype: torch.float32


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
import torch

# Initialize tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
dev_dataset = Dataset.from_pandas(dev_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_dev = dev_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_dev.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print("Datasets tokenized successfully!")
print(f"Tokenized train features: {list(tokenized_train.features.keys())}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Datasets tokenized successfully!
Tokenized train features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
from transformers import AutoModelForSequenceClassification

# Define the model with proper configuration for multi-label classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(emotion_labels),
    problem_type="multi_label_classification"
)

# Verify the model configuration
print(f"Model problem type: {model.config.problem_type}")
print(f"Number of labels: {model.config.num_labels}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model problem type: multi_label_classification
Number of labels: 28


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define metrics function for multi-label classification
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Apply sigmoid and threshold
    predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
    predictions = (predictions > 0.5).astype(int)

    # Convert labels to numpy array if they're tensors
    if hasattr(labels, 'numpy'):
        labels = labels.numpy()

    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='micro', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_total_limit=2,
    report_to='none',
)

In [ ]:
!wandb disabled


W&B disabled.


In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    compute_metrics=compute_metrics,
)

# Start training
print("Starting training...")
trainer.train()

# Save the model
trainer.save_model("emotion_classifier")
tokenizer.save_pretrained("emotion_classifier")
print("Model saved successfully!")

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.088700,0.088991,0.411537,0.540071,0.701930,0.438871
2,0.077600,0.082693,0.461850,0.579153,0.710995,0.488558
3,0.062100,0.085654,0.478621,0.588711,0.670404,0.524765


Model saved successfully!


In [ ]:
# First, let's load your saved model to make sure it works
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load your trained model
model = AutoModelForSequenceClassification.from_pretrained("emotion_classifier")
tokenizer = AutoTokenizer.from_pretrained("emotion_classifier")

# Make sure it's in evaluation mode
model.eval()

# Define emotion labels
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

# Create prediction function
def predict_emotions(text, threshold=0.3):
    # Tokenize input
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=128
    )

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits)

    # Get top emotions
    emotions = []
    for i, prob in enumerate(probs[0]):
        if prob > threshold:
            emotions.append((emotion_labels[i], float(prob)))

    # Sort by probability
    emotions.sort(key=lambda x: x[1], reverse=True)

    return emotions

# Test with some interesting examples
test_texts = [
    "I'm so excited about this project! It's going to be amazing!",
    "This is terrible and I hate it so much. What a waste of time.",
    "I feel so conflicted - happy for you but also nervous about the future",
    "That movie was so bad it's actually hilarious",
    "I'm not angry, just disappointed and a little sad",
    "WOW! This is incredible! I can't believe how good this is!",
    "I'm feeling anxious about the meeting tomorrow, but also optimistic",
    "This food is disgusting, I want my money back"
]

print("🧠 Testing Emotion Detection Model:\n")
for i, text in enumerate(test_texts, 1):
    emotions = predict_emotions(text)
    print(f"{i}. Text: {text}")
    print(f"   Emotions: {emotions}")
    print()

🧠 Testing Emotion Detection Model:

1. Text: I'm so excited about this project! It's going to be amazing!
   Emotions: [('excitement', 0.8305314779281616)]

2. Text: This is terrible and I hate it so much. What a waste of time.
   Emotions: [('disgust', 0.48984044790267944), ('anger', 0.35763049125671387), ('fear', 0.3464171588420868)]

3. Text: I feel so conflicted - happy for you but also nervous about the future
   Emotions: [('nervousness', 0.573256254196167)]

4. Text: That movie was so bad it's actually hilarious
   Emotions: [('amusement', 0.9386019110679626)]

5. Text: I'm not angry, just disappointed and a little sad
   Emotions: [('sadness', 0.8650633096694946)]

6. Text: WOW! This is incredible! I can't believe how good this is!
   Emotions: [('admiration', 0.905690610408783)]

7. Text: I'm feeling anxious about the meeting tomorrow, but also optimistic
   Emotions: [('optimism', 0.6423282623291016), ('nervousness', 0.31498920917510986)]

8. Text: This food is disgusting, I 

In [ ]:
# Evaluate on the official test set
test_results = trainer.evaluate(tokenized_test)
print("📊 Test Set Performance:")
print(f"Accuracy: {test_results['eval_accuracy']:.3f}")
print(f"F1 Score: {test_results['eval_f1']:.3f}")
print(f"Precision: {test_results['eval_precision']:.3f}")
print(f"Recall: {test_results['eval_recall']:.3f}")
print(f"Loss: {test_results['eval_loss']:.4f}")

📊 Test Set Performance:
Accuracy: 0.478
F1 Score: 0.590
Precision: 0.677
Recall: 0.524
Loss: 0.0849


In [ ]:
# See which emotions your model is good/bad at detecting
from sklearn.metrics import classification_report
import numpy as np

# Get predictions on test set
predictions = trainer.predict(tokenized_test)
pred_probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (pred_probs > 0.5).astype(int)
true_labels = predictions.label_ids

# Detailed classification report
print("📈 Detailed Performance by Emotion:")
print(classification_report(true_labels, pred_labels, target_names=emotion_labels, zero_division=0))

# Check emotion frequency
print("\n🎯 Emotion Frequency in Test Set:")
emotion_counts = true_labels.sum(axis=0)
for i, count in enumerate(emotion_counts):
    print(f"{emotion_labels[i]:<15}: {count} samples")

📈 Detailed Performance by Emotion:
                precision    recall  f1-score   support

    admiration       0.69      0.72      0.71       504
     amusement       0.79      0.83      0.81       264
         anger       0.57      0.43      0.49       198
     annoyance       0.47      0.22      0.30       320
      approval       0.51      0.34      0.41       351
        caring       0.50      0.37      0.43       135
     confusion       0.53      0.36      0.43       153
     curiosity       0.55      0.46      0.50       284
        desire       0.66      0.42      0.51        83
disappointment       0.45      0.17      0.25       151
   disapproval       0.43      0.31      0.36       267
       disgust       0.65      0.36      0.46       123
 embarrassment       0.81      0.35      0.49        37
    excitement       0.64      0.36      0.46       103
          fear       0.74      0.71      0.72        78
     gratitude       0.95      0.90      0.92       352
         gri

In [ ]:
# Test on some real tweet-like content
tweets = [
    "OMG this is the best day ever!!! 🎉🙌",
    "Traffic is awful today. Why is everyone driving so badly? 😠",
    "Not sure how I feel about this... mixed emotions tbh 🤔",
    "So grateful for my friends and family ❤️",
    "This meeting could have been an email 😴",
    "Anxious about the presentation but ready to crush it! 💪"
]

print("🐦 Testing on Tweet-like Content:\n")
for tweet in tweets:
    emotions = predict_emotions(tweet)
    print(f"Tweet: {tweet}")
    print(f"Emotions: {[e[0] for e in emotions]}")
    print()

🐦 Testing on Tweet-like Content:

Tweet: OMG this is the best day ever!!! 🎉🙌
Emotions: ['joy', 'excitement']

Tweet: Traffic is awful today. Why is everyone driving so badly? 😠
Emotions: ['disgust']

Tweet: Not sure how I feel about this... mixed emotions tbh 🤔
Emotions: ['confusion']

Tweet: So grateful for my friends and family ❤️
Emotions: ['gratitude']

Tweet: This meeting could have been an email 😴
Emotions: ['neutral', 'optimism']

Tweet: Anxious about the presentation but ready to crush it! 💪
Emotions: ['nervousness']



In [ ]:
# Test on LinkedIn comments
comments = [
    "innovative and impactful, hope it gets noticed and put into use🫡❤️",
    "Excellent work !",
    "Excellent work my brother Mohamed 👏🏻👏🏻👏🏻👏🏻👏🏻",
    "Well done Mohamed",
    "Very well done, Mohamed !!",
    "Très bon travail , félicitations.",
    "Félicitations Mohamed ! 🎉",
    "Excellent work 🤝 , excellent english, where are my credits at 🤡",
    "Excellent work Mohamed 🙌",
    "Impressive work Mohamed Love the focus on real administrative needs that's where most students struggle 👏",
    "Very good job brother, keep up the amazing work ❤️",
    "Very well done, Mohamed",
    "Absolutely amazing👏🏻👏🏻👏🏻👏🏻",
    "Excellent travail Mohamed 👏🏽👌🏾",
    "Je trouve cela très intéressant, Mohamed😎 continue frère 🫶🏽",
    "Excellent work 👏",
    "Excellent work Mohamed",
    "Impressive 🔥",
    "Very helpful",
    "Great work, my brother Mohamed Saber i found it really interesting",
    "Great initiative! 👏 This has real potential to enhance student support and streamline university communication. Looking forward to seeing how the project evolves, keep up the great work bro ❤️ !"
]

print("💼 Testing on LinkedIn Comments:\n")
for i, comment in enumerate(comments):
    emotions = predict_emotions(comment)
    print(f"Comment {i+1}: {comment}")
    print(f"Emotions: {[e[0] for e in emotions]}")
    print()

💼 Testing on LinkedIn Comments:

Comment 1: innovative and impactful, hope it gets noticed and put into use🫡❤️
Emotions: ['optimism']

Comment 2: Excellent work !
Emotions: ['admiration']

Comment 3: Excellent work my brother Mohamed 👏🏻👏🏻👏🏻👏🏻👏🏻
Emotions: ['admiration']

Comment 4: Well done Mohamed
Emotions: ['admiration']

Comment 5: Very well done, Mohamed !!
Emotions: ['admiration']

Comment 6: Très bon travail , félicitations.
Emotions: ['neutral']

Comment 7: Félicitations Mohamed ! 🎉
Emotions: ['neutral']

Comment 8: Excellent work 🤝 , excellent english, where are my credits at 🤡
Emotions: ['admiration']

Comment 9: Excellent work Mohamed 🙌
Emotions: ['admiration']

Comment 10: Impressive work Mohamed Love the focus on real administrative needs that's where most students struggle 👏
Emotions: ['admiration', 'love']

Comment 11: Very good job brother, keep up the amazing work ❤️
Emotions: ['admiration']

Comment 12: Very well done, Mohamed
Emotions: ['admiration']

Comment 13: Abso